# Assignment 2.2: Genome assembly

# Learning outcomes
To get:
- Familiarity with $k$-mer-based genome assembly (i.e. de Bruijn graphs)
- A practical understanding of contig assembly based on unitigs (including the benefits of error correction)
- A basic knowledge about evaluating assembly results
- An understanding that non-trivial algorithms are required for great practical results (not always, but often)

# Contents
### 1. Basics on de Bruijn graphs
    1.1 Construction
    1.2 Traversal
    1.3 Visualization

### 2. Contig assembly
    2.1 Unitig assembly
    2.2 Assembly of high-throughput sequencing reads

### 3. Evaluating assembly results
    3.1. Numerical statistics
    3.2. Visual statistics

### 4. Error handling
    4.1. Validation
    4.2. Correction

### 5. OPTIONAL: Gap filling

---
$^\ast$The algorithms and datasets in this assignment assume a simplified scenario: the genome is single stranded (i.e., no reverse complements), it is small (ebola virus, 20K bases), every $k$-mer in the genome is sequenced. Moreover, reads have length 100, coverage 20x, and their error rate is 0.75%.

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 0</strong></h3>

At the end of the assignment, write here how many hours it took to complete it (approximately). Write also general feedback about the assignment. What was difficult, what was easy, what was interesting, what was not, what other tasks / topics you would have liked to have in it?

#### Your answers here ...
---

---
- What is your background? (CS/Bio)

#### Your answers here ...
---

# 1. Basics on de Bruijn graphs

In this section you will develop the main functionality of a de Bruijn graph. 

The de Bruijn graph will be represented as a Python dictionary containing the $k$-mers of the graph. (Given the large amount of data from a sequencing experiment, state-of-the-art genome assemblers use more advanced data structures, and an optimized implementation in C++ for example.)

## 1.0 A note on edge-centric / node-centric de Bruijn graphs

Recall that during the lecture we added an edge between two $k$-mers if there was also a $(k+1)$-mer supporting the edge. This is called an *edge-centric* de Bruijn graph. 

In this assignment, for simplicity, we will work instead with *node-centric* de Bruijn graphs, where we add an edge whenever there is a suffix-prefix overlap of length $k-1$ between the two $k$-mers (i.e., the $(k+1)$-mer spelled by the edge doesn't need to be present in the reads).

## 1.1. Construction

We will begin with a simple example: we construct the de Bruijn graph based on all the $k$-mers in a *single* string `genome`. We also initialize a list of all DNA characters.

In [ ]:
dnaBases = ['A','C','G','T']
genome = 'ACGATCGTTC'

The de Bruijn graph built from all $3$-mers of the above string ``genome`` will be this one: <img src='data/testDBGraph.svg'>

Let's construct it with the function `build_dbGraph_from_string` below. We loop through the $k$-mers of the input string and add them to `dbGraph`. In this assignment, by "adding", we mean that we add the corresponding $k$-mer to the dictionary `dbGraph` with value 1. We'll use those values latter for describing *abundancies*.

In [ ]:
def build_dbGraph_from_string(string, order):
    dbGraph = dict()
    for i in range(len(string) - (order-1)):
        kmer = string[i:i+order]
        dbGraph[kmer] = 1
    return dbGraph

k = 3 # This is the order of our test de Bruijn graph 
test_dbGraph = build_dbGraph_from_string(genome, k)

print(test_dbGraph)

## 1.2. Traversal

We now implement a function that will give us the out-neighbors of a given node (i.e., of a given $k$-mer) in the de Bruijn graph.

Consider first three helper functions on strings (and $k$-mers).

In [ ]:
def nextKmer(kmer, newBase):
    return kmer[1:] + newBase

def prevKmer(kmer, newBase):
    return newBase + kmer[:-1]

def merge_strings(s, kmer):
    return s + kmer[-1]

print('Next kmer for "GAT" and "C" is:', nextKmer('GAT', 'C'))
print('Previous kmer for "GAT" and "C" is:', prevKmer('GAT', 'C'))
print('The merge of "ACGAT" and "ATC" is:', merge_strings('ACGAT','ATC'))

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 1</strong></h3>

Explain what the three functions above do.

---
#### Your answer here ...
---

We now implement a function which, given a de Bruijn graph `graph` and a string `kmer`, returns the list of out-neighbors of `kmer`. This  works by enumerating through the list `dnaBases`, considers the function `nextKmer` applied to `kmer` over each such DNA base, and checks whether this possible next kmer is an element of the dictionary `graph`. To check if a kmer `s` is an element in the keys of the dictionary `graph`, we use `if s in graph:`.

In [ ]:
def out_neighbors(graph, kmer):
    n = list() # The list of out-neighbors
    for base in dnaBases:
        nKmer = nextKmer(kmer, base)
        if nKmer in graph:
            n.append(nKmer)
            
    return n

print(out_neighbors(test_dbGraph, 'ACG'))

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 2</strong></h3>

It is your job now to implement the symmetric function returning the list of in-neighbors of a given `kmer` in `test_dbGraph`. **Hint**: Use exactly the same code as in `out_neighbors` but using the function `prevKmer` instead.

In [ ]:
def in_neighbors(graph, kmer):
    n = list() # The list of in-neighbors
    # Your code here ...
            
    return n

print(in_neighbors(test_dbGraph, 'CGA'))

## 1.3. Visualization

Having the dictionary `dbGraph` and the function `out_neighbors`, we have the basic functionality of a de Bruijn graph. As a sanity check of the implementation, we can visualize our de Bruijn graph constructed from `genome`. We do this with the **graphviz** package (using the `dot` representation). Make sure the graph you obtain is the same as the one shown above.


In [ ]:
from graphviz import Digraph

def get_dot_representation(graph):
    dot = Digraph()
    dot.graph_attr['rankdir'] = 'LR' # Display the graph in landscape mode

    for key in graph:
        kmer = key
        dot.node(kmer) 
        for neighbor in out_neighbors(graph, kmer):
            dot.edge(kmer, neighbor) 
     
    return dot
    

dot_representation = get_dot_representation(test_dbGraph)
dot_representation

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 3</strong></h3>

Explain how the above code works.

---
#### Your answer here ...
---

# 2. Contig assembly

## 2.1. Unitig assembly

Recall that in class we said that a node is the first node of some unitig if its number of in-neighbors is different from $1$ or if its number of out-neighbors is different from $1$. Also, the start of a unitig must have at least $1$ out-neighbor.

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 4</strong></h3>

Implement the following function. **Hint**: Use the functions `in_neighbors` and `out_neighbors` implemented above.

In [ ]:
def is_start_of_unitig(graph, kmer):
    # Returns True if and only if kmer is the first node of some unitig in graph
    is_start = False
    # Your code here ...
    
    return is_start

Recall that we say that a path $U = (v_1,v_2,\dots,v_t)$ is a *unitig* if either
- $t = 2$, that is, $U = (v_1,v_2)$ is a single edge, or
- $t > 2$ and for all $i \in \{2,\dots,t-1\}$, it holds that $v_i$ has only one in-neighbor (in this case, $v_{i-1}$) and only one out-neighbor (in this case, $v_{i+1}$).

Given a node `firstKmer` and one of its out-neighbors, `secondKmer`, the following function traverses the path starting with the edge `(firstKmer,secondKmer)` as long as it is a unitig. While doing so, it extends the current string `unitig` by merging to it the unique out-neighbor `nextKmer`. See the image below, where the nodes of the unitig are shown in green.

<img src='data/unitig-1.png'>

In [ ]:
def assemble_unitig_starting_with(graph, firstKmer, secondKmer):
    unitig = merge_strings(firstKmer,secondKmer)
    o_neighbors = out_neighbors(graph,secondKmer)
    i_neighbors = in_neighbors(graph,secondKmer)
    while len(o_neighbors) == 1 and len(i_neighbors) == 1:
        nextKmer = o_neighbors[0]
        unitig = merge_strings(unitig,nextKmer)
        o_neighbors = out_neighbors(graph,nextKmer)
        i_neighbors = in_neighbors(graph,nextKmer)
    return unitig

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 5</strong></h3>

Explain in your own words the code above.

---
#### Your answer here ...
---

We now have the two main ingredients of the algorithm finding all unitigs of the graph. Let's put them to work. We enumerate through all keys (i.e. $k$-mers) of the dictionary graph (`for kmer in graph:`) , for each such $k$-mer we test if it is the start of some unitig, and if so, we enumerate all its out-neighbors, and call the function `assemble_unitig_starting_with` with the $k$-mer and its out-neighbor as parameters. See the image below. 

<img src='data/unitig-2.png'>

In [ ]:
def assemble_unitigs(graph):
    unitigs = list()

    for kmer in graph:
        if is_start_of_unitig(graph, kmer):
            for neighbor in out_neighbors(graph, kmer):
                unitig = assemble_unitig_starting_with(graph, kmer, neighbor)
                unitigs.append(unitig)
    
    return unitigs

Check that the unitigs you get for `test_dbGraph` are the correct ones, by looking at the visualization of the graph.

In [ ]:
test_unitigs = assemble_unitigs(test_dbGraph)
print(test_unitigs)
dot_representation = get_dot_representation(test_dbGraph)
dot_representation

## 2.2. Assembly of high-throughput sequencing reads

The graph `test_dbGraph` was built from a single genomic string. We now build a graph from the reads in a *FASTQ* file. First, we implement the following function that gets the reads. We remove the new line character from the end of each line (with `str.strip()`), and for convention, we convert all characters in the reads to uppper case (with `str.upper()`).

In [ ]:
def get_reads(filePath):
    reads = list() # The list of strings that will store the reads (the DNA strings) in the FASTQ file at filePath
    fastqFile = open(filePath, 'r') 
    fastqLines = fastqFile.readlines() 
    fastqFile.close()

    for lineIndex in range(1, len(fastqLines), 4):
        line = fastqLines[lineIndex]
        reads.append(line.strip().upper())
        
    return reads

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 6</strong></h3>

Explain what is the purpose of the line `for lineIndex in range(1, len(fastqLines), 4):`.

---
#### Your answer here ...
---

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 7</strong></h3>

You now have to write a function equivalent to `build_dbGraph_from_string`, which however builds the graph from a collection of strings.

In [ ]:
def build_dbGraph_from_reads(reads, order):
    dbGraph = dict()
    # Your code here ...
            
    return dbGraph

With the above two functions in hand, we can build our de Bruijn graph from the reads in `reads.fastq`. For now, we will use the order `k = 15`.

In [ ]:
k = 15
reads = get_reads('data/reads.fastq')
dbGraph = build_dbGraph_from_reads(reads, k)

unitigs = assemble_unitigs(dbGraph)

print('Assembled', len(unitigs), 'unitigs')

# 3. Evaluating assembly results

With our unitigs in hand, we now try to get an idea of how good are these assembly results. 

## 3.1. Numerical statistics

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 8</strong></h3>

Implement a function that prints the mean, median, minimum and maximum values of the list of unitig lenths. Print also the number of unitigs, and the total length of all unitigs. *Hint*: use the Python package **statistics**, and the Python built-in functions **min(list)**, **max(list)** and **sum(list)**.

In [ ]:
import statistics

def print_basic_statistics(unitigs):
    mean = 0
    median = 0
    minimum = 0
    maximum = 0
    totalLength = 0
    # Your code here ...
    
    print('Mean unitig length:', mean)
    print('Median unitig length:', median)
    print('Min unitig length:', minimum)
    print('Max unitig length:', maximum)
    print('Number of unitigs:', len(unitigs))
    print('Total length of unitigs:', totalLength)

print_basic_statistics(unitigs)

Another measure of how contiguous an assembly is, is the N50 value. This is defined as the greatest unitig length $N$ such that the unitigs of length $\geq N$ have sum of lengths at least 50% of all the lengths of all unitigs.

In [ ]:
def print_N50(unitigs):
    sorted_unitigs = sorted(unitigs, reverse = True, key=lambda unitig: len(unitig)) # the list of unitigs is now sorted by length from large to small

    totalLength = sum([len(unitig) for unitig in sorted_unitigs])

    partialLength = 0
    for unitig in sorted_unitigs:
        partialLength += len(unitig)
        if partialLength >= 0.5 * totalLength:
            print('N50 value:', len(unitig))
            return

    
print_N50(unitigs)

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 9</strong></h3>

The above definition (before the previous code cell) of the N50 measure is a bit hard to understand. Explain what N50 represents based on the above code that computes it.

---
#### Your answer here ...
---

## 3.2. Visual statistics

Finally, to get a better understanding of the contiguity of the unitigs, we do a scatter plot of their lengths as follows. 

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 10</strong></h3>

Explain how the scatter plot below is obtained and what each dot represents.

---
#### Your answer here ...
---

In [ ]:
import matplotlib.pyplot as pyplot

def plot_lengths(unitigs):
    pyplot.style.use('seaborn-whitegrid')
    
    unitig_lengths = [len(unitig) for unitig in unitigs]
    unitig_lengths.sort(reverse = True)
    
    pyplot.plot(unitig_lengths, 'o', color='black') # We plot list
    
plot_lengths(unitigs)

# 4. Error handling

**The above assembly results are quite dismal: unitigs are only slightly longer than the value of `k` used in the assembly!**

On second thought, recall that the reads contain sequencing errors: these introduce erroneous $k$-mers in the graph, and thus introduce branches inside unitigs. 

We'll now see how to deal with these.

## 4.1. Validation

To convince us of this fact, suppose for the purpose of this assignment that we also have available the *true genome* from which the reads were sequenced. **This is not true in practice, since we are actually trying to figure out this genome!** However, development and evaluation of genome assemblers usually follows this pattern (since we now have many available genomes), as a validation and benckmarking step.

Let's now align all unitigs to this true genome, and see how many actually align. The ones that do not thus contain $k$-mers with errors.

First, we read the true genome from the FASTA file `data/genome.fasta`.

In [ ]:
def get_genome(filePath):
    genome = str()
    
    fastaFile = open(filePath, 'r') 
    
    fastaFile.readline() # Read header line
    fastaLines = fastaFile.readlines()
    fastaFile.close()

    for line in fastaLines:
        genome += line.strip().upper()
    
    return genome

trueGenome = get_genome('data/genome.fasta')

# Print the first 200 characters of trueGenome, for sanity check
print(trueGenome[:200])

Since we cannot call an actual read aligner in this Jupyter notebook, we will use the Python function `str.find(substring)`, which returns the position of the first occurence of ``substring`` inside ``str``, or ``-1`` if no such occurrence exists.

In [ ]:
def align(genome, unitigs):
    notFound = 0 # This will store the number of unitigs not found in genome

    for unitig in unitigs:
        pos = genome.find(unitig)
        if pos == -1:
            notFound += 1
    
    return notFound
    
notFound = align(trueGenome, unitigs)

print('Unitigs not aligning:', notFound)
print('Proportion of unitigs not aligning:', notFound / len(unitigs))

**Wow, a signficant amount of unitigs do not align!** This is a strong indication that the reads contain errors, and that we must filter these out. 

## 4.2. Correction

The main error correction idea used in the assembly field is to decide on an `abundance` threshold (which will depend on read coverage and error rate), such that any $k$-mer which appears less than `abundance` times in the reads is not added as a node in the de Bruijn graph. 

We will now implement this approach and will again assemble unitigs, just as before, in this new de Bruijn graph. We will then compare the unitig lengths before and after this correction step. 

Consider the following analogue of `build_dbGraph_from_reads`, which now takes into account also the parameter `abundance`.

In [ ]:
# The abundance threshold we will use throughout
a = 3

def build_dbGraph_from_reads_abundance(reads, order, abundance):
    dbGraph = dict()

    count = dict()
    for read in reads:
        for i in range(len(read) - (order - 1)):
            kmer = read[i:i+order]
            count[kmer] = count.get(kmer,0) + 1 # count.get(kmer,0) returns count[kmer], if kmer is a key in count, and 0 otherwise
    for kmer in count:
        if count[kmer] >= abundance:
            dbGraph[kmer] = abundance
    
    return dbGraph

corrected_dbGraph = build_dbGraph_from_reads_abundance(reads, k, a)
print('Nodes in the graph:', len(corrected_dbGraph))

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 11</strong></h3>

Explain how the above code works.

---
#### Your answer here ...
---

Let's now assemble the unitigs of `corrected_dbGraph` and compute the statistics about its unitigs.

In [ ]:
corrected_unitigs = assemble_unitigs(corrected_dbGraph)

print_basic_statistics(corrected_unitigs)
print_N50(corrected_unitigs)
plot_lengths(corrected_unitigs)
print('Unitigs not aligning:', align(trueGenome, corrected_unitigs))

**Things are now significatly different!**

We managed to obtain only **34 unitigs**, and half of all their total length comes from unitigs of **length 2518 or more (N50 metric)**. Moreover, only **8** of them do not align to the true benchmark genome. 

**A simple correction approach makes such a big difference.**

### Which `k` and `abundance` to choose?

Finally, let's consider the tradeoff between `k` and `abundance`. The code below assembles the unitigs of the de Bruijn graph for several values of `k` and several values of `a`. For each pair `(k,a)`, it computes how many unitigs are assembled, and how many of them do not align to the true reference.

<span style="color:red"><strong>Run the code below and wait several seconds for it to finish!</strong></span>

In [ ]:
import pandas
from IPython.display import display

results = list()
abundance_values = [1,2,3,4,5,6]
k_values = [15,21,27,39,51]

for a_value in abundance_values:
    a_results = list()
    for k_value in k_values:
        print(f'Getting results for a:{a_value}, k:{k_value}. Please wait ...')
        temp_dbGraph = build_dbGraph_from_reads_abundance(reads, k_value, a_value)
        temp_unitigs = assemble_unitigs(temp_dbGraph)
        temp_notFound = align(trueGenome, temp_unitigs)
        a_results.append((len(temp_unitigs),temp_notFound))
    results.append(a_results)

row_labels = [f'a:{a_value}' for a_value in abundance_values]
col_labels = [f'k:{k_value}' for k_value in k_values]
   
print('\nDone, here are the results. The format of each cell is (#unitigs, #unitigs not aligning).\n')
display(pandas.DataFrame(data=results, index=row_labels, columns=col_labels, dtype=None, copy=False))

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK 12</strong></h3>

Explain the results of the above table. Where are the best results, where are the worse results, **why**? 

How would you choose the values of `k` and `a` in practice (when you don't have the true genome to guide the choice)?

---
#### Your answers here ...
---

# OPTIONAL: 4. Gap filling

Gap filling is one of the last steps on a genome assembly pipeline. 

The main steps of an assembly pipeline (e.g. with short reads) are: 
1. Error correction
2. Contig assembly
3. Scaffolding
4. Gap filling

So far in this assignment, we've covered the first two steps. 

In the scaffolding step, paired-end information is used to connect the contigs together. More precisely, if the first read in a pair aligns to a contig $A$, and the second read in the pair aligns to a contig $B$, then this is an indication of how contigs $A$ and $B$ appear in the genome. 

In fact, by inspecting the precise alignment of the two reads inside each contig (i.e., their starting positions inside the contig), and knowing the approximate length of genomic distance between the two reads in the pair, then one can also infer the distance between contigs $A$ and $B$ in the genome. 

There are many ways to perform scaffolding (as mentioned during the lecture), and we will not cover scaffolding here. 

---

Instead, we will focus on **Step 4. Gap filling.**

First, we read a single scaffold from a FASTA file. For simplicity of code, we assume that this scaffold is made up only of two contigs, and sequence of characters `N` between them (denoting the unknown bases).

The length of this sequence of `N` equals the length of the unknown genomic sequence between the contigs. 

Let's read in the single scaffold from file `data/scaffolds.fasta`.

In [ ]:
def get_scaffold(filePath):
    string = str()
    
    fastaFile = open(filePath, 'r') 
    fastaFile.readline() # Read header line
    fastaLines = fastaFile.readlines() 
    fastaFile.close()
    
    for line in fastaLines:
        string += line.strip().upper()
    
    pos_firstN = string.find('N')
    pos_lastN = string.rfind('N')
    firstContig = string[:pos_firstN]
    lastContig = string[pos_lastN + 1:]
    gap_length = pos_lastN - pos_firstN + 1
    
    return firstContig, lastContig, gap_length

firstContig, lastContig, gap_length = get_scaffold('data/scaffolds.fasta')
print('Gap length:', gap_length)

We now try to see how the graph looks like between `firstContig` and `lastContig`. (More specifically, between the $k$-mer made up of the last `k` characters of `firstContig` and the $k$mer made up of the first `k` characters of `lastContig`.) That is, we check how many simple paths (without repeated nodes) there are between these two kmers. Each of these is a possible way to fill the gap.

<span style="color:red"><strong>Run the code below and wait up to one minute for it to finish!</strong></span>

In [ ]:
import networkx as nx

def convert_to_networkx_graph(graph):
    arcs = list()
    
    for kmer in graph:
        for o_neighbor in out_neighbors(graph,kmer):
            arcs.append((kmer,o_neighbor))
    
    return nx.DiGraph(arcs)
    
networkx_graph = convert_to_networkx_graph(corrected_dbGraph)
print('Converted the de Bruijn graph to nextworkx format')

fh = open("data/dbGraph.edgelist",'wb')
nx.write_edgelist(networkx_graph, fh)
fh.close()

print(f'k = {k}')

source_kmer = firstContig[-k:]
target_kmer = lastContig[:k]

print(f'Finding all simple paths (without repeated nodes) between source_kmer:{source_kmer} and target_kmer:{target_kmer}')
print('Please wait up to one minute...')
all_paths = list(nx.all_simple_paths(networkx_graph, source=source_kmer, target=target_kmer))

print('Found simple paths of the following lengths:')
for index, path in enumerate(all_paths):
    print(f'Path {index}, length:{len(path)}')

**Bingo!** We have a gap of length $624$, and we found a path of length $640$ (in addition to two other longer paths). In fact, this path is a perfect match for our gap length (you are required to explain why below). 

As such, this is a strong indication that the genomic string corresponding to path is the correct way to fill the gap between the two scaffolds.

Consider the function below.

In [ ]:
def convert_path_to_string(path, k):
    string = str()
    
    for kmer in path[1:-k]:
        string = merge_strings(string, kmer)
        
    return string

solutionPath = all_paths[0]
gapString = convert_path_to_string(solutionPath, k)

print(len(gapString))

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK</strong></h3>

Explain why the code above gives a string of length matching exactly the gap length (i.e., the number of `N` characters in the scaffold).

---
#### Your answer here ...
---

Finally, we fill the gap in the scaffold. 

We also align it to the true genome, in order to verify our assumption that the filling path is correct.

In [ ]:
filledScaffold = firstContig + gapString + lastContig

print('Filled scaffold aligns to genome:', align(filledScaffold, trueGenome) == 0)

### Note on practicality

The purpose of the above code is to show you that even if a gap cannot be filled just by doing unitig assembly, further genomic content can be revealed by adding in the paired-end information, **even in the form of a single number, the gap length**. 

However, practical gap filler do not work exactly in the above manner. Instead, they try to include the expected gap length in the search for the filling path.

<h3 style='background:yellow;padding:10px;color:black'><strong>TASK</strong></h3>

Explain what are the drawbacks of using a code very similar to the above (even if one has an equivalent C++ implementation) when doing gap filling in practice.

---
#### Your answer here ...
---